2024-02-21 19:56:16.406015: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 19:56:17.094786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 19:56:17.095026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 19:56:17.228105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 19:56:17.408226: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 19:56:17.409432: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 5.6414 - accuracy: 0.4286
Epoch 2/10
1/1 [==============================] - 0s 65ms/step - loss: 132.0299 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 72ms/step - loss: 41.4161 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 76ms/step - loss: 0.7099 - accuracy: 0.9286
Epoch 5/10
1/1 [==============================] - 0s 61ms/step - loss: 7.3822 - accuracy: 0.5714
Epoch 6/10
1/1 [==============================] - 0s 59ms/step - loss: 6.2920 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 60ms/step - loss: 2.5419 - accuracy: 0.5714
Epoch 8/10
1/1 [==============================] - 0s 60ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 59ms/step - loss: 2.1673 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 55ms/step - loss: 0.5054 - accuracy: 0.7857


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import librosa

# Load audio file and extract features
def load_and_extract_features(*audio_paths):
    mfccs_list = []
    pitch_list = []
    formants_list = []

    for audio_path in audio_paths:
        y, sr = librosa.load(audio_path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        pitch = np.mean(librosa.yin(y, fmin=50, fmax=200))
        formants = librosa.effects.harmonic(y)

        mfccs_list.append(mfccs)
        pitch_list.append(pitch)
        formants_list.append(formants)

    return mfccs_list, pitch_list, formants_list

# Define audio paths
human_audio_paths = [
    'Human-voice1.mp3',
    'Human-voice2.mp3',
    'Human-voice3.mp3',
    'Human-Test-Voice.mp3',
    'Human-voice4.mp3',
    'Human-voice5.mp3',
    'Human-voice6.mp3'
]

ai_audio_paths = [
    'voicemail-354.mp3',
    'voicemail-355.mp3',
    'voicemail-356.mp3',
    'voicemail-357(2).mp3',
    'voicemail-357(1).mp3',
    'voicemail-358.mp3',
    'voicemail-358(1).mp3'
]

# Extract features for humans and AIs
human_mfccs, human_pitch, human_formants = load_and_extract_features(*human_audio_paths)
ai_mfccs, ai_pitch, ai_formants = load_and_extract_features(*ai_audio_paths)

# Truncate arrays to a fixed length
max_length = 500
human_mfccs_processed = [arr[:, :max_length] if arr.shape[1] >= max_length else np.pad(arr, ((0, 0), (0, max_length - arr.shape[1])) , mode='constant') for arr in human_mfccs]
ai_mfccs_processed = [arr[:, :max_length] if arr.shape[1] >= max_length else np.pad(arr, ((0, 0), (0, max_length - arr.shape[1])) , mode='constant') for arr in ai_mfccs]

# Concatenate processed arrays
data = np.concatenate((human_mfccs_processed, ai_mfccs_processed), axis=0)

# Create labels (0 for human, 1 for AI)
labels = np.array([0] * len(human_mfccs_processed) + [1] * len(ai_mfccs_processed))

# Reshape data to fit the input shape of the model
data = data.reshape((-1, 13, max_length, 1))

# Model Definition
num_mfcc = data.shape[1]
num_frames = data.shape[2]

model = models.Sequential([
    layers.Input(shape=(num_mfcc, num_frames, 1)),  # Input shape for MFCC features
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Model Training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(data, labels, epochs=10, batch_size=32)

# Inferencing
def classify_audio(audio_path):
    y, sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_processed = mfccs[:, :max_length] if mfccs.shape[1] >= max_length else np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])) , mode='constant')
    features = mfccs_processed.reshape((1, 13, max_length, 1))
    predicted_label = model.predict(features)
    if predicted_label > 0.5:
        return "This voice is a Bot voice"
    else:
        return "This voice is a Human voice"

#Testing
audio_path = 'voicemail-354.mp3'  
result = classify_audio(audio_path)
print(result)


Epoch 1/10
1/1 [==============================] - 1s 868ms/step - loss: 1.6606 - accuracy: 0.7143
Epoch 2/10
1/1 [==============================] - 0s 66ms/step - loss: 146.1852 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 69ms/step - loss: 50.4178 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 66ms/step - loss: 4.3409 - accuracy: 0.7143
Epoch 5/10
1/1 [==============================] - 0s 66ms/step - loss: 23.7489 - accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 64ms/step - loss: 1.5890 - accuracy: 0.9286
Epoch 7/10
1/1 [==============================] - 0s 63ms/step - loss: 9.1819e-16 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 66ms/step - loss: 1.3451e-12 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 62ms/step - loss: 1.8988e-04 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 68ms/step
This voice is a Bot voice
